In [ ]:
B = 1000
results = matrix(nrow=B,ncol=39)
set.seed(123)
n1=100####sample size for diseased subjects and it depends on scenarios
n2=100####sample size for healthy subjects and it depends on scenarios
mean1 = 0 #### mean for diseased subjects and it depends on scenarios
sd1 =1#### standard deviation for diseased subjects and it depends on scenarios
mean2=0.5####mean for healthy subjects and it depends on scenarios
sd2=1#### standard deviation  for healthy subjects and it depends on scenarios
find.closest <- function(M, crit) {
  M[which(is.na(M))] = Inf
  mindiff = min(abs(M - crit))
  which((M == crit + mindiff) | (M == crit - mindiff), 
        arr.ind = T)
}
crit_value = c(0.69,0.77,0.85) #### AUCROC values for effect sizes
#n=n1+n2
i=1
library(extraDistr)
library(dplyr)
library(tidyr)
library(stringr)
library(fossil)
library(cutpointr)
require(pracma)

start.time <- Sys.time()
while(i<=B){
  
  tryCatch({
    
    d_free = data.frame( rnorm(n=n1,mean=mean1, sd=sd1))
    d_diseased = data.frame(rnorm(n=n2, mean = mean2, sd = sd2))
    colnames(d_free) = "X"
    colnames(d_diseased) = "X"
    healthy=data.frame(rep(0,n1))
    diseased=data.frame(rep(1,n2))
    colnames(healthy)="y"
    colnames(diseased)="y"
    data=data.frame(cbind(data.frame(rbind(d_free,d_diseased)),data.frame(rbind(healthy,diseased))))
    test = data$X
    ref = data$y
    data$ID = seq(1:nrow(data))
    ###AUC ROC#######
    data_roc<-data.frame(cbind(data$y,data$X))
    roccurve<-function(data,increasing=FALSE,healthy){
      library(precrec)
      library(ggplot2)
      library(plotROC)
      library(OptimalCutpoints)
      library(pROC)
      library(Epi)
      library(ROCR)
      data<-data[complete.cases(data),]
      p=ncol(data)
      n=nrow(data)
      colnames(data)[p-1]<-"status"
      colnames(data)[p]<-"X"
      k=healthy  ####0 or 1
      basicplot<-ggplot(data, aes(d = data[,p-1], m = data[,p]))
      if(increasing==FALSE){
        basicplot<-basicplot+geom_roc(labels = T,increasing = FALSE)+
          style_roc(theme = theme_grey, xlab = "1 - Specificity",ylab="Sensitivity")
        basicplot<-basicplot+annotate("text", x = .75, y = .25, 
                                      label = paste("AUC =", round(calc_auc(basicplot)$AUC, 3)))
        opt1<-optimal.cutpoints(X = "X", status = "status", tag.healthy = k,
                                methods = "Youden", data = data, pop.prev = NULL,
                                control = control.cutpoints(), ci.fit = TRUE, 
                                conf.level = 0.95, trace = FALSE,direction=">")
        result<-summary(opt1)
      }else{
        basicplot<-basicplot+geom_roc(n.cuts=n,labels = F,increasing = TRUE)+
          style_roc(theme = theme_grey, xlab = "1 - Specificity",ylab="Sensitivity")
        basicplot<-basicplot+annotate("text", x = .75, y = .25, label = paste("AUC =", 
                                                                              round(calc_auc(basicplot)$AUC, 3)))
        opt1<-optimal.cutpoints(X = "X", status = "status", tag.healthy = k,
                                methods = "Youden", data = data, pop.prev = NULL,
                                control = control.cutpoints(), ci.fit = TRUE, 
                                conf.level = 0.95, trace = FALSE,direction="<")
        result<-summary(opt1)}
      results<-list("Roc Results"=result,basicplot)
      return(results)
    }
    k=roccurve(data_roc,increasing = T,healthy = 0)
    auc_roc =k$`Roc Results`$Youden$Global$measures.acc$AUC
    
    
    #####Grey zone approach
    Se = seq(0.01,0.5,0.01)
    Sp = seq(0.01,0.5,0.01)
    all_combinations = crossing(Se,Sp, .name_repair = "unique")
    all_combinations = data.frame(all_combinations)
    
    
    opt1 = cutpointr(data, x = X , class=y,method = oc_youden_kernel,pos_class=1)
    sensiti = opt1$sensitivity
    spef = opt1$specificity
    all_combinations$sensitivities_range = (all_combinations$Se+sensiti)
    all_combinations$specifities_range = (all_combinations$Sp+spef)
    all_combinations = all_combinations[all_combinations$sensitivities_range<=1,]
    all_combinations = all_combinations[all_combinations$specifities_range<=1,]
    all_combinations$lrp = all_combinations$sensitivities_range/(1-all_combinations$specifities_range)
    all_combinations$lrn = (1-all_combinations$sensitivities_range)/(all_combinations$specifities_range)
    
    gz_results = matrix(nrow=nrow(all_combinations),ncol=3)
    for(j in 1:nrow(all_combinations)){
        gz = g_zone(ref, test, criterion.values = c(all_combinations[j,6], 
                                                       all_combinations[j,5]),return.all = T)
        gz_lim = c(gz$thresholds[1],gz$thresholds[length(gz$thresholds)])
        lt = gz_lim[1]
        ut = gz_lim[2]
        data$grup_greyzone<-ifelse(data$X<lt,0, 
                                   ifelse (data$X>ut,1,2))
        gz_table = table(data$grup_greyzone,data$y)
        
        library(caret)
        
        gz_cm = confusionMatrix(gz_table[c(1,2),],positive="1")
        
        gz_accuracy = gz_cm$overall[1]
        
        gz_sens = gz_cm$byClass [1]
        
        gz_spef = gz_cm$byClass [2]
        
        X = c(gz_sens)
        
        Y = 1-c(gz_spef)
        
        names(X) = NULL
        
        names(Y) = NULL
        
        roc_df = as.data.frame(matrix(c(0,0,X,Y,1,1),nrow=3,byrow=T))
        
        X1 = roc_df$V1
        
        Y1 = roc_df$V2

        AUC= trapz(x=Y1,y=X1)
        
        gz_results[j,1:3] = matrix(c(lt,ut,AUC),ncol=3)
    }


    gz_results = data.frame(gz_results)
    
    gz_l = gz_results[find.closest(gz_results$X3,crit_value[1]-0.01),]##crit_value[1] will cahnge based on scenario
    
    gz_u = gz_results[find.closest(gz_results$X3,crit_value[1]+0.01),]##crit_value[1] will cahnge based on scenario
    
    gz = rbind(gz_l,gz_u)
    
    gz$dif = gz$X2-gz$X1
    
    gz_f = gz[which.min(gz$dif),]
    
    gz_low = c(gz_f$X1)
    
    names(gz_low) =NULL
    
    gz_up = gz_f$X2
    
    names(gz_up) =NULL
    
    gz_dif = gz_up - gz_low
    
    data$grup_greyzone<-ifelse(data$X<gz_low,0, 
                              ifelse (data$X>gz_up,1,2))
    
    gz_table = table(data$grup_greyzone,data$y)
    
    library(caret)
    
    gz_cm = confusionMatrix(gz_table[c(1,2),],positive="1")
    
    gz_accuracy = gz_cm$overall[1]
    
    gz_sens = gz_cm$byClass [1]
    
    gz_spef = gz_cm$byClass [2]
    
    gz_diseased = gz_table[3,2]
    
    gz_control = gz_table[3,1]
    
    X = c(gz_sens)
    
    Y = 1-c(gz_spef)
    
    names(X) = NULL
    
    names(Y) = NULL
    
    roc_df = as.data.frame(matrix(c(0,0,X,Y,1,1),nrow=3,byrow=T))
    
    X1 = roc_df$V1
    Y1 = roc_df$V2
    
    gz_AUC= trapz(x=Y1,y=X1)
    
    gz_results_all = c(gz_low,gz_up,gz_dif,gz_diseased,gz_control,gz_AUC,gz_accuracy,gz_sens,gz_spef)
    
####Uncertain Interval Approach
    library(UncertainInterval)
    
    Se_ui = seq(0.51,0.65,0.01)
    
    Sp_ui = seq(0.51,0.65,0.01)
    
    all_combinations_ui = crossing(Se_ui,Sp_ui, .name_repair = "unique")
    
    all_combinations_ui = data.frame(all_combinations_ui)
    
    ui_results = matrix(nrow=nrow(all_combinations_ui),ncol=3)
    
    for(j in 1:nrow(all_combinations_ui)){
    
      ui = ui.binormal(ref, test,UI.Se = all_combinations_ui[j,1] ,
                                UI.Sp = all_combinations_ui[j,2])
      lt = ui$solution[1]
      
      ut = ui$solution[2]
      
      data$grup_ui<-ifelse(data$X<lt,0, 
                                 ifelse (data$X>ut,1,2))
      
      ui_table = table(data$grup_ui,data$y)
      
      library(caret)
      
      ui_cm = confusionMatrix(ui_table[c(1,2),],positive="1")
      
      ui_accuracy = ui_cm$overall[1]
     
      ui_sens = ui_cm$byClass [1]
      
      ui_spef = ui_cm$byClass [2]
      
      X = c(ui_sens)
      
      Y = 1-c(ui_spef)
      
      names(X) = NULL
      
      names(Y) = NULL
      
      roc_df = as.data.frame(matrix(c(0,0,X,Y,1,1),nrow=3,byrow=T))
      
      X1 = roc_df$V1
      
      Y1 = roc_df$V2
      
      AUC= trapz(x=Y1,y=X1)
      
      ui_results[j,1:3] = matrix(c(lt,ut,AUC),ncol=3)
    }
    
    ui_results =data.frame(ui_results)
    
    ui_l = ui_results[find.closest(ui_results$X3,crit_value[1]-0.01),]##crit_value[1] will cahnge based on scenario
    
    ui_u = ui_results[find.closest(ui_results$X3,crit_value[1]+0.01),]##crit_value[1] will cahnge based on scenario
    
    ui = rbind(ui_l,ui_u)
    
    ui$diff = ui$X2 - ui$X1
      
    ui_res = ui[which.min(ui$diff),]
    
    ui_low= c(ui_res$X1)
    
    names(ui_low) =NULL
    
    ui_up = ui_res$X2
    
    names(ui_up) =NULL
    
    ui_dif = ui_up - ui_low
    
    data$ui_group<-ifelse(data$X<ui_low,0, 
                               ifelse (data$X>ui_up,1,2))
    
    ui_table = table(data$ui_group,data$y)
    
    library(caret)
    
    ui_cm = confusionMatrix(ui_table[c(1,2),],positive="1")
    
    ui_accuracy = ui_cm$overall[1]
    
    ui_sens = ui_cm$byClass [1]
    
    ui_spef = ui_cm$byClass [2]
    
    ui_diseased = ui_table[3,2]
    
    ui_control = ui_table[3,1]
    
    X = c(ui_sens)
    
    Y = 1-c(ui_spef)
    
    names(X) = NULL
    
    names(Y) = NULL
    
    roc_df = as.data.frame(matrix(c(0,0,X,Y,1,1),nrow=3,byrow=T))
    
    X1 = roc_df$V1
    Y1 = roc_df$V2
    
    ui_AUC= trapz(x=Y1,y=X1)
    
    ui_results_all = c(ui_low,ui_up,ui_dif,ui_diseased,ui_control,ui_AUC,ui_accuracy,ui_sens,ui_spef)
    
    

###Proposed methods Kernel based
    pm<-info3.kb_area_sim(ref,test)
    
    pm_kb_ul = pm$thresholds$UL
    
    pm_kb_ll = pm$thresholds$LL
    
    pm_diff = pm_kb_ul - pm_kb_ll
    
    data$grup_pm_kb = ifelse(data$X< pm$thresholds$LL,0, 
                             ifelse (data$X>pm$thresholds$UL,1,2))
    
    pm_table = table(data$grup_pm_kb,data$y)
    
    pm_cm = confusionMatrix(pm_table[c(1,2),],positive="1")
    
    pm_accuracy = pm_cm$overall[1]
    
    pm_sens = pm_cm$byClass [1]
    
    pm_spef = pm_cm$byClass [2]
    
    
    X = c(pm_sens)
    
    Y = 1-c(pm_spef)
    
    names(X) = NULL
    
    names(Y) = NULL
    
    roc_df = as.data.frame(matrix(c(0,0,X,Y,1,1),nrow=3,byrow=T))
    
    X1 = roc_df$V1
    Y1 = roc_df$V2
    
    pm_AUC= trapz(x=Y1,y=X1)
    
    
    pm_zone_control=pm_table[3,1]
    
    pm_zone_diseased = pm_table[3,2]
    

    pm_kb = c(pm_kb_ll,pm_kb_ul,pm_diff, pm_zone_diseased,pm_zone_control,
             pm_AUC, pm_accuracy,pm_sens,pm_spef)

    
    
    pm_dist<-info3_area_sim(ref,test)
    
    pm_dist_ll = pm_dist$thresholds$LL
    
    pm_dist_ul = pm_dist$thresholds$UL
    
    pm_dist_diff =  pm_dist_ul -  pm_dist_ul 
    
    pm_cut_dist = pm_dist$thresholds[3]
    
    data$grup_pm_dist = ifelse(data$X<pm_dist$thresholds$LL,0, 
                             ifelse (data$X>pm_dist$thresholds$UL,1,2))
    
    pm_table_dist = table(data$grup_pm_dist,data$y)
    
    pm_cm_dist = confusionMatrix(pm_table_dist[c(1,2),],positive="1")
    
    pm_accuracy_dist = pm_cm_dist$overall[1]
    
    pm_sens_dist = pm_cm_dist$byClass [1]
    
    pm_spef_dist = pm_cm_dist$byClass [2]
    
    X = c(pm_sens_dist)
    
    Y = 1-c(pm_spef_dist)
    
    names(X) = NULL
    
    names(Y) = NULL
    
    roc_df = as.data.frame(matrix(c(0,0,X,Y,1,1),nrow=3,byrow=T))
    
    X1 = roc_df$V1
    Y1 = roc_df$V2
    
    pm_dist_AUC = trapz(x=Y1,y=X1)
    
    
    pm_zone_control_dist = pm_table_dist[3,1]
    
    pm_zone_diseased_dist = pm_table_dist[3,2]
    

    
    
    pm_dist = c(pm_dist_ll,pm_dist_ul,pm_dist_diff,pm_zone_diseased_dist,
                pm_zone_control_dist, pm_dist_AUC, pm_accuracy_dist,pm_sens_dist,pm_spef_dist)
    
    

    names(gz_results_all) =NULL
    names(ui_results_all)=NULL
    names(pm_kb)= NULL
    names(pm_dist) = NULL
    names(auc_roc) = NULL
    
    
    results[i,1:39] = matrix(c(auc_roc,gz_results_all,ui_results_all,pm_kb,pm_dist),ncol=39)
    
    
    
    i = dim(na.omit(results))[1]
    i=i+1
  },error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
}
end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

colnames(results) = c("auc_roc","auc_roc_l","auc_roc_u","gz_ll","gz_ul",
                      "gz_diff","gz_diseased","gz_control","gz_AUC",
                      "gz_accuracy","gz_sensitivity","gz_specificity",
                      "ui_ll", "ui_ul","ui_diff","ui_diseased","ui_control",
                      "ui_AUC","ui_accuracy", "ui_sensitivity","ui_specificity",
                      "pm_kb_ll","pm_kb_ul","pm_kb_diff","pm_kb_diseased",
                      "pm_kb_control","pm_kb_AUC","pm_kb_accuracy", 
                      "pm_kb_sensitivity","pm_kb_specificity", 
                      "pm_dist_ll","pm_dist_ul","pm_dist_diff","pm_dist_diseased",
                      "pm_dist_control","pm_dist_AUC","pm_dist_accuracy", 
                      "pm_dist_sensitivity","pm_dist_specificity"
                     )

summary(results)
results = data.frame(results)




